In [1]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import numpy as np
from keras.models import model_from_json
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report,ConfusionMatrixDisplay
from tensorflow.keras.layers import *
from tensorflow.keras import layers , models, optimizers
from tensorflow.keras.models import Sequential, Model

In [2]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [26]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.2),
    layers.RandomContrast(0.2)
    
])

In [30]:
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# create model structure
emotion_model = Sequential()

emotion_model.add(data_augmentation)

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(BatchNormalization())

emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())

emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(BatchNormalization())

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())

emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(BatchNormalization())

emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
emotion_model.add(BatchNormalization())

emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(BatchNormalization())

emotion_model.add(Flatten())
emotion_model.add(Dense(256, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(BatchNormalization())

emotion_model.add(Dense(128, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(BatchNormalization())

emotion_model.add(Dense(7, activation='softmax'))
# You can call model.build() to build the model's architecture, or create a dummy input to build it.
emotion_model.build((None, 48, 48, 1))

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [31]:
# Train the neural network/model
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=5,        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/5
448/448 [==============================] - 281s 620ms/step - loss: 0.4000 - accuracy: 0.8680 - val_loss: 1.1498 - val_accuracy: 0.6436
Epoch 2/5
448/448 [==============================] - 330s 738ms/step - loss: 0.3099 - accuracy: 0.9004 - val_loss: 1.2526 - val_accuracy: 0.6247
Epoch 3/5
448/448 [==============================] - 301s 671ms/step - loss: 0.2528 - accuracy: 0.9220 - val_loss: 1.2801 - val_accuracy: 0.6458
Epoch 4/5
448/448 [==============================] - 323s 722ms/step - loss: 0.2178 - accuracy: 0.9304 - val_loss: 1.3759 - val_accuracy: 0.6385
Epoch 5/5
448/448 [==============================] - 337s 752ms/step - loss: 0.1788 - accuracy: 0.9445 - val_loss: 1.4538 - val_accuracy: 0.6300


In [33]:
emotion_model.evaluate(validation_generator)

113/113 [==============================] - 11s 98ms/step - loss: 1.4549 - accuracy: 0.6297


[1.4549182653427124, 0.6297018527984619]

In [32]:
model_json = emotion_model.to_json()
with open("models/emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('models/emotion_model.h5')